<a href="https://colab.research.google.com/github/cagBRT/timeSeries/blob/main/11d_CNN_LSTMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CNN-LSTMs**

A CNN model can be used in a hybrid model with an LSTM backend where the <br>
- CNN is used to interpret subsequences of input that<br>
- are provided as a sequence to an LSTM model to interpret. <br><br>

**This hybrid model is called a CNN-LSTM**



Combining the advantages of convolutional neural networks 
- (CNN) that can extract effective features from the data, and 
- long short-term memory (LSTM) which can not only find the interdependence of data in time series data, but also automatically detect the best mode suitable for relevant data, 

this method can effectively improve the accuracy of stock price forecasting.<br>
[Hindawi](https://www.hindawi.com/journals/complexity/2020/6622927/)

In [ ]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils.vis_utils import plot_model

In [ ]:
def split_sequence(sequence, n_steps):
  X, y = list(), list()
  for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the sequence
    if end_ix > len(sequence)-1:
     break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)


In [ ]:
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

In [ ]:
n_steps = 4
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
n_features = 1
n_seq = 2
n_steps = 2
X = X.reshape((X.shape[0], n_seq, n_steps, n_features))

In [ ]:
model = Sequential()
model.add(TimeDistributed(Conv1D(64, 1, activation='relu'), input_shape=(None, n_steps,
n_features))) 
model.add(TimeDistributed(MaxPooling1D())) 
model.add(TimeDistributed(Flatten())) 
model.add(LSTM(50, activation='relu')) 
model.add(Dense(1)) 
model.compile(optimizer='adam', loss='mse')

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
model.fit(X, y, epochs=500, verbose=0)

**Make prediction**<br>
We are expecting 100

In [ ]:
x_input = array([60, 70, 80, 90])
x_input = x_input.reshape((1, n_seq, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)